In [2]:
# test for iexfinance API
from iexfinance.stocks import Stock

appl = Stock('AAPL', token="sk_5949b3f3377c4d278e80fa93394b9bda")

quote = appl.get_quote()
print(quote)

{'symbol': 'AAPL', 'companyName': 'Apple, Inc.', 'primaryExchange': 'NASDAQ', 'calculationPrice': 'close', 'open': 210, 'openTime': 1567171800675, 'close': 208.74, 'closeTime': 1567195200578, 'high': 210.45, 'low': 207.2, 'latestPrice': 208.74, 'latestSource': 'Close', 'latestTime': 'August 30, 2019', 'latestUpdate': 1567195200578, 'latestVolume': 21087548, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexLastUpdated': None, 'delayedPrice': 208.74, 'delayedPriceTime': 1567196100013, 'extendedPrice': 208.15, 'extendedChange': -0.59, 'extendedChangePercent': -0.00283, 'extendedPriceTime': 1567295993472, 'previousClose': 209.01, 'previousVolume': 21007652, 'change': -0.27, 'changePercent': -0.00129, 'volume': 21087548, 'iexMarketPercent': None, 'iexVolume': None, 'avgTotalVolume': 29851979, 'iexBidPrice': None, 'iexBidSize': None, 'iexAskPrice': None, 'iexAskSize': None, 'marketCap': 943333633200, 'peRatio': 17.64, 'week52High': 233.47, 'week52Low': 142, 'ytdChange': 0.320519, 'last

In [ ]:
# Deploy to the Wechat by WXPY
from wxpy import *
bot = Bot()
my_friend = bot.friends().search('三丫')[0]

In [3]:
from iexfinance.stocks import Stock
# use API to get the stock information
def get_stock_info(stock_symbol):
    stock = Stock(stock_symbol,token="sk_5949b3f3377c4d278e80fa93394b9bda")
    try:
        quote = stock.get_quote()
    except:
        return ""
    else:
        template = "{} in {} have latest price {} updated in {}"
        return template.format(quote["companyName"],quote['primaryExchange'],str(quote['latestPrice']),quote['latestTime'])
print(get_stock_info("IBM"))
# return the query informations
def query_stock_info(stock_symbols):
    query_info = []
    for stock in stock_symbols:
        info = get_stock_info(stock)
        if info is not "":
            query_info.append(info)
        if len(query_info) == 0:
            return None
    return query_info
print(query_stock_info("IBM"))
# format the information   
def format_info(query_info):
    result = ""
    for info in query_info:
        result = result+"\n"+info
    return result
print(format_info(query_stock_info(["IBM"])))

International Business Machines Corp. in New York Stock Exchange have latest price 135.53 updated in August 30, 2019
['Intelsat SA in New York Stock Exchange have latest price 20.68 updated in August 30, 2019', 'Barnes Group, Inc. in New York Stock Exchange have latest price 44.85 updated in August 30, 2019', "Macy's, Inc. in New York Stock Exchange have latest price 14.76 updated in August 30, 2019"]

International Business Machines Corp. in New York Stock Exchange have latest price 135.53 updated in August 30, 2019


In [4]:
# extract the stock symbol by regex, return an empty list if not matched.
import re
text = "I Want to check the stock for APPL, IBM and TSLE"
def extract_entities_re(text):
    pattern = re.compile(r"([\w]+[A-Z]+[\w]+)|(^[A-Z]+$)")
    results = pattern.findall(text)
    intent_list = []
    for item in results:
        for intent in item:
            if intent is not "":
                intent_list.append(intent.strip())
    return intent_list
print(extract_entities_re(text))

['APPL', 'IBM', 'TSLE']


In [5]:
def extract_entities(text):
    return extract_entities_re(text)

In [6]:
# extract the entities with negation entities return entities, neg_entities
def extract_all_entities(text):
    entities = extract_entities(text)
    neg_policy = ["n't","not"]
    # get the index of the final character of each entities
    ends_index = sorted([text.index(e)+len(e) for e in entities])
    # initialise a list to store sentence chuncks
    chunks = []
    start = 0
    for end in ends_index:
        chunks.append(text[start:end])
        start = end
    pos_entities = []
    neg_entities = []
    for chunk in chunks:
        for entity in entities:
            if entity in chunk:
                if "not" in chunk or "n't" in chunk:
                    neg_entities.append(entity)
                else:
                    pos_entities.append(entity)
    return pos_entities,neg_entities
text = "I want the stock for AAPL but not IBM"
print(extract_all_entities(text))         

(['AAPL'], ['IBM'])


In [7]:
# define the robot behaviors (behavior:String -> response:[String])
robot_behaviors = {
    "default":["I'm sorry, I am not quite understand you :("],
    "greet":["Hello my friend, I am your stock assitant, I can help you solve stock problems","Hi my friend, what can I help you?"],
    "explain_functions":["I can help you to check the stock information","I can help you to check your stock, please tell me the stock symbol."],
    "ask_more_entities":["Please tell me the stock symbol you what to search."],
    "confirm_with_stock_entities":["No problem, wait me for a second"],
    "confirm_without_stock_entities":["Sure, could you give me the stock symbol please?"],
    "answer_stock_info":["Here is the information about the stock {},do you what to search another stock?"],
    "ask_another_search":["Do you what to search another stock?"],
    "search_failed":["Sorry, I cannot find the information about the stock, do you want another try?"],
    "farewell_ok":["Good by, have a nice day!"],
    "farewell_failed":["Bye, I am sorry that I can't help you for that T_T"]
}

In [8]:
# define the robot state
INIT = 0
STOCK_SEARCH = 1  
FAREWELL = 2 

# define the pending state
REQUIRE_ENTITIES = 0
ENTITIES_OK = 1
DATA_OK = 2
DATA_FAILED = 3



In [9]:
# define state transition policy
policy = {
    (INIT,None,None):(INIT,False,robot_behaviors["default"]),
    (INIT,None,"greet"):(INIT,False,robot_behaviors["greet"]),
    (INIT,None,"ask_functions"):(INIT,False,robot_behaviors["explain_functions"]),
    (INIT,REQUIRE_ENTITIES,"stock_search"):(STOCK_SEARCH,True,robot_behaviors["confirm_without_stock_entities"]),
    (INIT,ENTITIES_OK,"stock_search"):(STOCK_SEARCH,True,robot_behaviors["confirm_with_stock_entities"]),
    (STOCK_SEARCH,REQUIRE_ENTITIES,None):(STOCK_SEARCH,False,robot_behaviors["ask_more_entities"]),
    (STOCK_SEARCH,ENTITIES_OK,None):(STOCK_SEARCH,True,robot_behaviors["confirm_with_stock_entities"]),
    (STOCK_SEARCH,ENTITIES_OK,"stock_search"):(STOCK_SEARCH,True,robot_behaviors["confirm_with_stock_entities"]),
    (STOCK_SEARCH,DATA_OK,None):(STOCK_SEARCH,False,robot_behaviors["answer_stock_info"]),
    (STOCK_SEARCH,DATA_OK,"stock_search"):(STOCK_SEARCH,False,robot_behaviors["answer_stock_info"]),
    (STOCK_SEARCH,DATA_OK,"confirm"):(STOCK_SEARCH,False,robot_behaviors["ask_more_entities"]),
    (STOCK_SEARCH,DATA_OK,"negative"):(INIT,False,robot_behaviors["farewell_ok"]),
    (STOCK_SEARCH,DATA_FAILED,None):(STOCK_SEARCH,True,robot_behaviors["search_failed"]),
    (STOCK_SEARCH,DATA_FAILED,"confirm"):(STOCK_SEARCH,False,robot_behaviors["ask_more_entities"]),
    (STOCK_SEARCH,DATA_FAILED,"negative"):(INIT,False,robot_behaviors["farewell_failed"]),
    (FAREWELL,None,"farewell"):(INIT,False,robot_behaviors["farewell_ok"])
    
}


In [10]:
# judge the pending stage
def pending(state,intent,current_entities,current_data):
    pendingState = None
    if state is INIT and intent != "stock_search":
        pendingState = None
        current_entities = []
        current_data = []
    elif state is INIT and intent == "stock_search":
        if len(current_entities) != 0:
            pendingState = ENTITIES_OK
        else:
            pendingState = REQUIRE_ENTITIES
    elif current_data is None:
        pendingState = DATA_FAILED
        current_data = []
    elif state is STOCK_SEARCH and len(current_entities) == 0 and len(current_data) is 0:
        pendingState =  REQUIRE_ENTITIES
    elif state is STOCK_SEARCH and len(current_entities) > 0 and current_data is not None: #and len(current_data) is 0
        pendingState = ENTITIES_OK
    elif state is STOCK_SEARCH and len(current_data) is not 0:
        pendingState = DATA_OK
    elif state is STOCK_SEARCH and len(current_entities) > 0 and current_data is None :
        pendingState = DATA_FAILED
    elif state is FAREWELL:
        pendingState = None
        current_entities = []
        current_data = []
        
    return pendingState
            

In [11]:
# execute the policy
import random
def execute_policy(state,intent,current_entities,current_data):
    pendingState = pending(state,intent,current_entities,current_data)
    if intent == "confirm" or intent == "negative":
        print("clear data")
        current_entities = []
        current_data = []
    try:
        next_state, isInternal, behaviors = policy[(state,pendingState,intent)]
        if "{}" in random.choice(behaviors):
            print("bot: "+random.choice(behaviors).format(format_info(current_data)))
            my_friend.send(random.choice(behaviors).format(format_info(current_data)))
        else:
            print("bot: "+random.choice(behaviors))
            my_friend.send(random.choice(behaviors))
    except:
        if intent == "farewell":
            next_state,current_entities,current_data = execute_policy(FAREWELL,intent,current_entities,current_data)
        else:
            next_state,current_entities,current_data = execute_policy(state,None,current_entities,current_data)
    else:
        if isInternal:
            if pendingState is ENTITIES_OK:
                current_data = query_stock_info(current_entities)
                if current_data is not None:
                    current_entities = []
                next_state,current_entities,current_data =  execute_policy(next_state,None,current_entities,current_data)
    return next_state,current_entities,current_data


In [1]:
# Import necessary modules
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

# Create a trainer that uses this config
trainer = Trainer(config.load("data/config_spacy.yml"))

# Load the training data
training_data = load_data('data/testData2.json')

# Create an interpreter by training the model
interpreter = trainer.train(training_data)
def interpret (message):
    intent = interpreter.parse(message)["intent"]["name"]
    entities = interpreter.parse(message)["entities"]
    return intent, entities

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_

In [12]:
# Initial System state
state = INIT
current_entities = []
current_data = []

In [13]:
# define the response function
def respond(message):
    intent, entities = interpret(message)
    entities = extract_all_entities(message)[0]
    
    next_state,new_entities,data = execute_policy(state,intent,entities,current_data)
  
    return next_state,new_entities,data

In [ ]:
@bot.register(my_friend)
def reply_my_friend(msg):
    global state, current_entities,current_data
    state,current_entities,current_data = respond(msg.text)
    
    return ""

In [ ]:
print("debug:",state,"current_entities:",current_entities,"current_data:",current_data)

In [20]:
state,current_entities,current_data = respond("No")


clear data
bot: Good by, have a nice day!
